<a href="https://colab.research.google.com/github/697kiran/Internship_project/blob/main/Internship_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai tavily-python langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.5 MB/s eta 0:00:00


In [ ]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23


In [ ]:
!pip install langchain langchain-openai langchain-community tavily-python


In [ ]:
import re
from typing import Dict, Any, List
from google.colab import userdata
from langchain_community.tools.tavily_search import TavilySearchResults

class TavilyResearchSystem:
    def __init__(self, tavily_api_key):
        self.search_tool = TavilySearchResults(
            tavily_api_key=tavily_api_key,
            k=8,
            include_raw_content=True,
            search_depth="advanced"
        )

    def search(self, query: str) -> Dict[str, Any]:
        print(f"Searching for information on: {query}")

        try:
            search_results = self.search_tool.invoke(query)

            processed_results = []
            sources = []

            for i, result in enumerate(search_results):
                if isinstance(result, dict):
                    title = result.get('title', f'Result {i+1}')
                    url = result.get('url', 'No URL available')
                    content = result.get('content', 'No content available')

                    processed_results.append({
                        "title": title,
                        "url": url,
                        "content": content
                    })

                    sources.append(url)

            return {
                "query": query,
                "results": processed_results,
                "sources": sources
            }

        except Exception as e:
            print(f"Error during search: {str(e)}")
            return {
                "query": query,
                "error": str(e),
                "results": [],
                "sources": []
            }

    def organize_by_topic(self, search_data: Dict[str, Any]) -> Dict[str, List]:
        """Organize research results by topic"""
        topics = {}
        results = search_data.get("results", [])

        key_phrases = [
            "advancement", "breakthrough", "development", "research",
            "technology", "innovation", "discovery", "application",
            "method", "technique", "algorithm", "hardware", "software"
        ]

        for result in results:
            title = result['title'].lower()
            content = result['content'].lower()

            detected_topics = [phrase for phrase in key_phrases
                              if phrase in title or phrase in content[:200]]

            topic = detected_topics[0].capitalize() if detected_topics else "General"
            topics.setdefault(topic, []).append(result)

        return topics

    def generate_research_report(self, query: str) -> Dict[str, Any]:
        """Generate a complete research report"""
        search_data = self.search(query)

        if "error" in search_data:
            return {
                "query": query,
                "report": f"Research Error: {search_data['error']}",
                "sources": []
            }

        topics = self.organize_by_topic(search_data)

        report = f"# Research Report: {query}\n\n"

        sources_count = len(search_data.get('sources', []))
        report += f"## Summary\n"
        report += f"This report contains research findings on '{query}' from {sources_count} sources.\n\n"

        if not topics:
            report += "No relevant information was found.\n"
        else:
            for topic, results in topics.items():
                report += f"## {topic}\n\n"

                for result in results:
                    report += f"### {result['title']}\n"
                    report += f"**Source:** [{result['url']}]({result['url']})\n\n"

                    content = result['content'][:800]
                    if len(result['content']) > 800:
                        content += "..."

                    report += f"{content}\n\n"

        report += "## Sources\n\n"
        for i, url in enumerate(search_data.get("sources", [])):
            report += f"{i+1}. [{url}]({url})\n"

        return {
            "query": query,
            "report": report,
            "sources": search_data.get("sources", [])
        }


def main():
    try:
        import sys
        import subprocess
        print("Installing required packages...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "langchain-community", "tavily-python"])

        TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

        if not TAVILY_API_KEY:
            print("Error: Tavily API key not found in Colab secrets.")
            print("Please add your TAVILY_API_KEY to Colab secrets.")
            return

        research_system = TavilyResearchSystem(TAVILY_API_KEY)

        default_query = "What are the latest advancements in quantum computing?"
        user_query = input(f"Enter your research query (press Enter for default: '{default_query}'): ")
        query = user_query.strip() or default_query

        result = research_system.generate_research_report(query)

        from IPython.display import Markdown, display
        display(Markdown(result["report"]))

        print("\n===== SOURCES =====")
        for i, source in enumerate(result["sources"]):
            print(f"{i+1}. {source}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Make sure you have added your Tavily API key to Colab secrets.")

if __name__ == "__main__":
    main()

Installing required packages...
Enter your research query (press Enter for default: 'What are the latest advancements in quantum computing?'): 
Searching for information on: What are the latest advancements in quantum computing?


# Research Report: What are the latest advancements in quantum computing?

## Summary
This report contains research findings on 'What are the latest advancements in quantum computing?' from 5 sources.

## Advancement

### 10 Key Quantum Computing Breakthroughs in 2025 - trailyn.com
**Source:** [https://www.trailyn.com/10-key-quantum-computing-breakthroughs-in-2025/](https://www.trailyn.com/10-key-quantum-computing-breakthroughs-in-2025/)

These advancements are paving the way for more reliable quantum systems, bringing us closer to realizing their full potential.

5. Combined Quantum and AI Systems

Thanks to advancements in hardware and error correction, quantum computing is now merging with AI to tackle industrial challenges in new ways. By 2025, this combination led to impressive gains in efficiency and energy use. [...] 2025 has been a transformative year for quantum computing, with major advancements across hardware, error correction, AI integration, and quantum networks. Here's a quick summary of the most important breakthroughs shaping the future:

Quick Comparison of Key Developments [...] 10 Key Quantum Computing Breakthroughs in 2025

Explore the transformative breakthroughs in quantum computing from 2025, includi...

### The State of Quantum Computing in 2024: Innovations, Challenges, and ...
**Source:** [https://methodologists.net/Exploring-the-Transformative-Advancements-in-Quantum-Computing-and-Their-Global-Impact-in-2024](https://methodologists.net/Exploring-the-Transformative-Advancements-in-Quantum-Computing-and-Their-Global-Impact-in-2024)

Discover the latest breakthroughs in quantum computing as of late 2024, from advancements in hardware and algorithms to impacts on cybersecurity, AI integration, and future industry trends.

### The latest developments in quantum computing: A transformative frontier ...
**Source:** [https://www.openaccessgovernment.org/the-latest-developments-in-quantum-computing-a-transformative-frontier/187748/](https://www.openaccessgovernment.org/the-latest-developments-in-quantum-computing-a-transformative-frontier/187748/)

With advancements in hardware, algorithms, and integration with other technologies, quantum computing promises to reshape the technological landscape. The journey toward quantum supremacy may still have hurdles, but the progress made in recent years underscores its transformative potential.

### Latest Advancements in Quantum Computing Technology
**Source:** [https://quantaintelligence.ai/2024/10/12/technology/latest-advancements-in-quantum-computing-technology](https://quantaintelligence.ai/2024/10/12/technology/latest-advancements-in-quantum-computing-technology)

The rapid advancements in quantum computing raise important ethical concerns, particularly regarding data security and privacy. As quantum computers become more powerful, they could potentially break traditional encryption methods, posing significant risks to sensitive information.

## Breakthrough

### Quantum Computing in 2025: Breakthroughs, Challenges, and Future ...
**Source:** [https://www.imecreative.com/quantum-computing-in-2025-breakthroughs-challenges-and-future-prospects/](https://www.imecreative.com/quantum-computing-in-2025-breakthroughs-challenges-and-future-prospects/)

3. Microsoft's Majorana 1: A Potential Game Changer. Microsoft's quantum computing research has focused on developing topological qubits, and its Majorana 1 chip is a critical step toward fault-tolerant quantum computers. The company aims to integrate quantum capabilities into its Azure cloud services, enabling broader accessibility for developers and researchers.

## Sources

1. [https://www.trailyn.com/10-key-quantum-computing-breakthroughs-in-2025/](https://www.trailyn.com/10-key-quantum-computing-breakthroughs-in-2025/)
2. [https://methodologists.net/Exploring-the-Transformative-Advancements-in-Quantum-Computing-and-Their-Global-Impact-in-2024](https://methodologists.net/Exploring-the-Transformative-Advancements-in-Quantum-Computing-and-Their-Global-Impact-in-2024)
3. [https://www.openaccessgovernment.org/the-latest-developments-in-quantum-computing-a-transformative-frontier/187748/](https://www.openaccessgovernment.org/the-latest-developments-in-quantum-computing-a-transformative-frontier/187748/)
4. [https://quantaintelligence.ai/2024/10/12/technology/latest-advancements-in-quantum-computing-technology](https://quantaintelligence.ai/2024/10/12/technology/latest-advancements-in-quantum-computing-technology)
5. [https://www.imecreative.com/quantum-computing-in-2025-breakthroughs-challenges-and-future-prospects/](https://www.imecreative.com/quantum-computing-in-2025-breakthroughs-challenges-and-future-prospects/)



===== SOURCES =====
1. https://www.trailyn.com/10-key-quantum-computing-breakthroughs-in-2025/
2. https://methodologists.net/Exploring-the-Transformative-Advancements-in-Quantum-Computing-and-Their-Global-Impact-in-2024
3. https://www.openaccessgovernment.org/the-latest-developments-in-quantum-computing-a-transformative-frontier/187748/
4. https://quantaintelligence.ai/2024/10/12/technology/latest-advancements-in-quantum-computing-technology
5. https://www.imecreative.com/quantum-computing-in-2025-breakthroughs-challenges-and-future-prospects/
